Package install


In [2]:
#! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain langchain-mistralai
#! pip install beautifulsoup4
#! pip install python-dotenv

Load Enviorment variables

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

True

Langsmith

In [4]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_PROJECT'] = "Simple RAG"

API Keys

In [5]:
LANGCHAIN_API_KEY = os.environ['LANGCHAIN_API_KEY']
MISTAL_API_KEY = os.environ['MISTRAL_API_KEY']
#os.environ['OPENAI_API_KEY']
#os.environ['HF_TOKEN']

Rag imports

In [6]:
import bs4
import tiktoken
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai import MistralAIEmbeddings
from langchain.prompts import ChatPromptTemplate


USER_AGENT environment variable not set, consider setting it to identify your requests.


Loading


In [7]:
#Loading a document
#Function takes documents and loads them.
#The parse only can change, depends on the html on what you are trying to load
loader = WebBaseLoader (
    web_paths = ('https://en.wikipedia.org/wiki/Ultimate_Fighting_Championship',),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer('body')
    ),
)
blog_docs =loader.load()


Splitting

In [8]:
#Splitting the document in to chunks 
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 1000,
    chunk_overlap =200)

#Make splits
splits = text_splitter.split_documents(blog_docs)

Indexing - Vector Storage

In [9]:
#Uses MistralAIEmbeddings
#Embeds the chunks and saves them in the Chroma Vector DB
#Index

vectorstorage = Chroma.from_documents(documents=splits,
                                      embedding=MistralAIEmbeddings())
#Retriver object
retriver = vectorstorage.as_retriever()

c:\Users\nikol\Datamatiker\4.Semester\Python\SimpleRAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\nikol\Datamatiker\4.Semester\Python\SimpleRAG\.venv\Lib\site-packages\langchain_mistralai\embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


Retrieval

In [10]:
docs = retriver.get_relevant_documents("Who is light weight champion")

C:\Users\nikol\AppData\Local\Temp\ipykernel_920\3728727279.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriver.get_relevant_documents("Who is light weight champion")


In [11]:
len(docs)

4

Generation

In [12]:

#Prompt template with a context variable and a question variable
template = ''' Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=' Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

Define LLM

In [13]:
#LLM
llm = ChatMistralAI(model_name="ministral-8b-latest", temperature=0)

In [14]:
#Chain
#Connecting promt and llm togehter
chain = prompt | llm

In [15]:
#Run our chain
#Using a dictionary content and question maps to our expected keys in our template

chain.invoke({"context":docs,"question":"Who is the current heavy weight champion"})

AIMessage(content='The current heavyweight champion is Jon Jones.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 4677, 'total_tokens': 4686, 'completion_tokens': 9}, 'model': 'ministral-8b-latest', 'finish_reason': 'stop'}, id='run-e8aa33f3-e6d3-41f6-9d25-81464c60f43e-0', usage_metadata={'input_tokens': 4677, 'output_tokens': 9, 'total_tokens': 4686})

ANother way to chain:

In [16]:
#Another way of doing it, looks a bit more smoother.
#We invoke with our question, that gets passed trough to our Key: Question in our Dictionary
# And it will automaticly trigger our retriver, which returns documents.
#This is automated compared to above

rag_chain = (
    {"context": retriver,"question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What would be the best heavy weight matchup and why")

"Based on the provided context, the best heavyweight matchup would be between Jon Jones and Tom Aspinall. Here's why:\n\n1. **Current Champions**: Jon Jones is the current heavyweight champion, while Tom Aspinall is the interim heavyweight champion. This makes it a title matchup.\n\n2. **Record and Skills**: Both fighters have impressive records and skills. Jon Jones has a record of 28–1 (1 NC) with 15 wins by knockout, while Tom Aspinall has a record of 15–3 with 10 wins by knockout. Both are known for their striking and grappling abilities.\n\n3. **Recent Performances**: Jon Jones has been inactive for a while, but he has shown his dominance in the past. Tom Aspinall has been active and has been performing well, including a win over Ciryl Gane.\n\n4. **Interest and Hype**: A matchup between these two would generate a lot of interest and hype, as both are highly respected and skilled fighters.\n\nSo, a matchup between Jon Jones and Tom Aspinall would be the best heavyweight matchup ba